# Import Packages

In [1]:
import numpy as np
import random
from nupack import *

# Set Up Functions

In [2]:
def structure_to_bp(structure):
    """
    Converts dot parens structure to dictionary of base pairs
    
    Args:
        structure (str): dot-parens structure notation
    
    Returns:
        bp (dict): dictionary of base pair indices
    """
    open_parens = [] # List of open parentheses
    bp = {} # Dict of base pairs
    for i, x in enumerate(structure):
        if x == '(':
            open_parens.append(i)
        elif x == ')':
            open_index = open_parens.pop()
            bp[open_index] = i
            bp[i] = open_index
    return bp

def make_random_sequence(structure):
    """
    Takes a given structure and generates a random inverse 
    nucleotide sequence

    Args:
        structure (str): desired dot-parens structure notation

    Returns:
        sequence (str): generated random RNA sequence
    """
    bases = ['A', 'G', 'U', 'C']
    pairs = {'A':'U', 'G':'C', 'U':'A', 'C':'G'}
    sequence = ['_'] * len(structure)
    bp = structure_to_bp(structure)
    for i, x in enumerate(structure):
        if x == '.':
            sequence[i] = random.choice(bases)
        elif x == '(':
            sequence[i] = random.choice(bases)
            close_index = bp[i]
            sequence[close_index] = pairs[sequence[i]] 
    sequence = ''.join(sequence)
    return sequence

def nupack_analyze_sequence(sequence):
    """
    Takes given sequence and runs it through Nupack package to
    return the lowest energy RNA secondary structure
    
    Args:
        sequence (str): RNA sequence
    
    Returns:
        nupack_structure (str): predicted RNA structure from Nupack
            package
    """
    model = Model(material='rna', celsius=37) # Physical model
    A = Strand(sequence, name='A')
    tube = Tube({A: 1e-8}, complexes = SetSpec(max_size=1),
               name='Tube 1')
    results = tube_analysis(tubes=[tube], compute=['pairs', 'mfe'],
                            model=model)
    tube_result = results['(A)']
    nupack_structure = str(tube_result.mfe[0].structure)
    return nupack_structure

def mutate_random_position(mismatch, sequence, structure, constraints=None):
    """
    Mutates a random position in sequence from list of indices
    in mismatch list
    
    Args:
        mismatch (list): list of indices from which random position
            can be mutated
        sequence (str): RNA sequence
        structure (str): dot-parens notation of desired RNA structure

    Returns:
        mutated_sequence (str): mutated RNA sequence
    """
    bases = ['A', 'G', 'U', 'C']
    pairs = {'A':'U', 'G':'C', 'U':'A', 'C':'G'}
    bp = structure_to_bp(structure)
    mutate_position = random.choice(mismatch) # Question!
    sequence = list(sequence)
    sequence[mutate_position] = random.choice(bases)
    if mutate_position in bp:
        mutate_bp_position = bp[mutate_position]
        sequence[mutate_bp_position] = pairs[
            sequence[mutate_position]]
    mutated_sequence = ''.join(sequence)
    return mutated_sequence
        
def structure_differences(structure, nupack_structure):
    """
    Returns a list of indices that correspond to mismatched bases
    in ideal and predicted RNA structures
    """
    mismatch = [] # List of mismatched bases indices
    structure = list(structure)
    nupack_structure = list(nupack_structure)
    for i in range(len(structure)):
        if structure[i] != nupack_structure[i]:
            mismatch.append(i)
    return mismatch

def compare_mutate_sequence(random_sequence, structure, nupack_structure):
    """
    Compares the ideal structure to the predicted structure of
    generated random RNA sequence from Nupack
    Mutates RNA sequence randomly if structures don't match
    
    Args:
        random_sequence (str): random RNA sequence from 
            random_sequence function
        structure (str): desired dot-parens structure notation
        nupack_structure (str): dot-parens structure of random sequence
            predicted from Nupack package

    Returns:
        mutated_sequence (str):
    """
    mutated_mismatch = []
    mutated_sequence = random_sequence
#     if structure == nupack_structure:
#         print('The structures match!')
    if structure != nupack_structure:
#         print('The structures do not match!')
        mismatch = structure_differences(structure, nupack_structure)
        mutated_sequence = mutate_random_position(mismatch, random_sequence, structure)
        print('The mutated sequence is', mutated_sequence)
        mutated_nupack_structure = nupack_analyze_sequence(mutated_sequence)
        mutated_mismatch = structure_differences(structure, mutated_nupack_structure)
    return mutated_mismatch, mutated_sequence


In [29]:
def mutate_sequence_iterate(random_sequence, structure, nupack_structure, iterations):
    """
    Iteratively compares predicted structure from Nupack to ideal structure,
    mutates random sequence if mismatched, and adopts mutated sequence under
    certain conditions
    
    Args:
        random_sequence (str):
        structure (str):
        nupack_structure (str):
        iterations (int):

    Returns:
        final_sequence (str):
    """
    mismatch = structure_differences(structure, nupack_structure)
    for i in range(iterations):
        print('--------------------------------')
        print(f'Iteration {i}')
        print('--------------------------------')
        mutated_mismatch, mutated_sequence = compare_mutate_sequence(
            random_sequence, structure, nupack_structure)
        if len(mutated_mismatch) == 0: # Set stopping condition if structure matches
            random_sequence = mutated_sequence
            mismatch = mutated_mismatch
            print('The predicted structure is:', nupack_structure)
            print('Structure matches!')
            print('Final sequence is', random_sequence)
            break
        if len(mutated_mismatch) <= len(mismatch): # Question!
            random_sequence = mutated_sequence
            mismatch = mutated_mismatch
            nupack_structure = nupack_analyze_sequence(random_sequence)
            print('The predicted structure is:', nupack_structure)
            print('Structure is improved: Adopting sequence')
            print()
        elif len(mutated_mismatch) > len(mismatch):
            random_number = random.random()
            if random_number >= 0.95:
                random_sequence = mutated_sequence
                mismatch = mutated_mismatch
                nupack_structure = nupack_analyze_sequence(random_sequence)
                print('The predicted structure is:', nupack_structure)
                print('Structure is worse: Adopting sequence')
                print()
            else:
                print('The predicted structure is:', nupack_structure)
                print('Structure is worse: Not adopting sequence')
                print()
        if i == iterations - 1 and len(mismatch) > 0:
            print('Sequence does not converge to ideal sequence')
            print('Final sequence is', random_sequence)
            print('The predicted structure is:', nupack_structure)
    final_sequence = random_sequence
    return final_sequence

# Test Simple RNA Sequence

In [23]:
structure = '...((((.....))))..((.....)).(((.......)))'

In [24]:
random_sequence = make_random_sequence(structure)

In [25]:
print(random_sequence)

UACACGCAGCUGGCGUCGCUCUUACAGUCUGCUUAGCUCAG


In [26]:
nupack_structure = nupack_analyze_sequence(random_sequence)

In [27]:
print(nupack_structure)

.....(((((((............))).)))).........


In [30]:
mutate_sequence_iterate(random_sequence, structure, nupack_structure, 100)

--------------------------------
Iteration 0
--------------------------------
The mutated sequence is UACACGGAGCUGCCGUCGCUCUUACAGUCUGCUUAGCUCAG
The predicted structure is: .....(((((((............))).)))).........
Structure is worse: Not adopting sequence

--------------------------------
Iteration 1
--------------------------------
The mutated sequence is UACACGCAGCUGGCGUCGCUCUUACAGUCUACUUAGCUUAG
The predicted structure is: .....(((((((............))).)))).........
Structure is worse: Not adopting sequence

--------------------------------
Iteration 2
--------------------------------
The mutated sequence is UACACGCAGCUGGCGUCGGUCUUACACUCUGCUUAGCUCAG
The predicted structure is: .....(((((((............))).)))).........
Structure is worse: Not adopting sequence

--------------------------------
Iteration 3
--------------------------------
The mutated sequence is UACCCGCAGCUGGCGGCGCUCUUACAGUCUGCUUAGCUCAG
The predicted structure is: .....(((((((............))).)))).........
Structure is wo

The predicted structure is: ...(((((...))))).((((............))))....
Structure is improved: Adopting sequence

--------------------------------
Iteration 35
--------------------------------
The mutated sequence is UACAAGCUGCUGGCUUCGCUCUUACAGUCUUCUGGCCUAAG
The predicted structure is: ...(((((...))))).((((............))))....
Structure is worse: Not adopting sequence

--------------------------------
Iteration 36
--------------------------------
The mutated sequence is UACAAGCUGCUGGCUUCGCUCUUACAGUCUUCUGGGCUAAG
The predicted structure is: ...(((((...))))).((((............))))....
Structure is improved: Adopting sequence

--------------------------------
Iteration 37
--------------------------------
The mutated sequence is UACAAGCUGCUGGCUUCGCUCUUACAGUCUUCUGGGCUAAG
The predicted structure is: ...(((((...))))).((((............))))....
Structure is improved: Adopting sequence

--------------------------------
Iteration 38
--------------------------------
The mutated sequence is UACAAGCUGCUGG

The predicted structure is: ...((((.....))))............(((.......)))
Structure is worse: Not adopting sequence

--------------------------------
Iteration 67
--------------------------------
The mutated sequence is UACAAGCCGCUUGCUUCGAUCUUACAUUCCCUUGGGCUGGG
The predicted structure is: ...((((.....))))............(((.......)))
Structure is improved: Adopting sequence

--------------------------------
Iteration 68
--------------------------------
The mutated sequence is UACAAGCCGCUUGCUUCGCUCUUACAGUCCCUUGGGCUGGG
The predicted structure is: ...((((.....))))............(((.......)))
Structure is worse: Not adopting sequence

--------------------------------
Iteration 69
--------------------------------
The mutated sequence is UACAAGCCGCUUGCUUCGUUCUUACAAUCCCUUGGGCUGGG
The predicted structure is: ...((((.....))))............(((.......)))
Structure is worse: Not adopting sequence

--------------------------------
Iteration 70
--------------------------------
The mutated sequence is UACAAGCCGCU